<a href="https://colab.research.google.com/github/KAMANNAGARIBHAVITHAGNA/codesnippets/blob/main/bhavithoob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup
import smtplib
from email.mime.text import MIMEText
import time
import re
import os

def get_price(url):
    try:
        response = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
            'Accept-Language': 'en-US,en;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'DNT': '1',
            'Cache-Control': 'max-age=0'
        })
        soup = BeautifulSoup(response.content, 'html.parser')

        price_element = soup.select_one('.a-price-whole')  # Amazon price selector

        if price_element:
            price_text = price_element.text.strip()
            price_number = re.findall(r'[\d,]+', price_text)
            if price_number:
                price = float(price_number[0].replace(',', ''))
                return price
        return None
    except Exception as e:
        print(f"Error fetching price: {e}")
        return None

def send_email(subject, body, to_email):
    gmail_user = os.getenv('EMAIL_USER')  # Use environment variables
    gmail_password = os.getenv('EMAIL_PASS')

    if not gmail_user or not gmail_password:
        print("Email credentials not set. Please configure them securely.")
        return

    try:
        msg = MIMEText(body)
        msg['Subject'] = subject
        msg['From'] = gmail_user
        msg['To'] = to_email

        server = smtplib.SMTP_SSL('smtp.gmail.com', 465)
        server.login(gmail_user, gmail_password)
        server.sendmail(gmail_user, to_email, msg.as_string())
        server.quit()

        print("Email sent successfully!")
    except Exception as e:
        print(f"Error sending email: {e}")

def track_price(url, target_price, check_interval=3600):
    while True:
        current_price = get_price(url)

        if current_price is not None:
            print(f"Current price: {current_price}")

            if current_price <= target_price:
                subject = "BRO BHAVI CHEPTUNNADU VINNANDI PRODUCT PRICE TAGGINDI"
                body = f"""
                The price dropped to {current_price}!
                This is below your target price {target_price}.
                Check the product here: {url}
                """
                send_email(subject, body, 'dillibhavith@gmail.com')
                print("Email pampa!")
                break
            else:
                print("Price raledu/taggledhu, malli chusta.")
        else:
            print("Could not retrieve the current price, will try again.")

        print("Malli 1 hour tarvata check chesta!")
        time.sleep(check_interval)

if __name__ == "__main__":
    url = "https://www.amazon.in/realme-Wireless-Earbuds-Spatial-Charging/dp/B0DBGNDTY2/"
    target_price = 500
    print("Price tracking start chesam...")
    track_price(url, target_price)


Price tracking start chesam...
Current price: 1999.0
Price raledu/taggledhu, malli chusta.
Malli 1 hour tarvata check chesta!
